# Medir Perplexity sobre frases sueltas de clase positiva vs clase negativa usando el modelo fine tuneado vs el sin fine tunear.

La idea es:

1. Tomar frases sueltas del corpus de Shakespeare vs un corpus que no tenga nada que ver (algo sobre medicina por ejemplo).

2. Evaluar la perplexity promedio sobre las dos clases de frases sueltas en los dos modelos.

3. La hipótesis es que el modelo finetuneado va a tener menor perplexity en las frases de shakespeare vs el modelo sin fine tuning, y va a pasar lo opuesto sobre las frases que no tengan nada que ver.

Nos gustaría mostrar que nuestro modelo fine tuneado pasó de ser de uso general para tener un uso bien particular.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q -U torch transformers accelerate bitsandbytes

import torch
import random
import re
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer
import math

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 145.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170

In [ ]:
PATH_TXT_SHAKESPEARE = "/content/drive/MyDrive/StoryWriter/Data/RAW/obras_shakespeare.txt"
PATH_TXT_RANDOM      = "/content/drive/MyDrive/StoryWriter/Data/RAW/easy/general_english_corpus.txt"

PATH_MODELO_BASE     = "/content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-7b-instruct-v0.3"
PATH_MODELO_FINETUNE = "/content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-finetuneado(lora)"

In [ ]:
def limpiar_texto(texto):
    """Limpia saltos de línea excesivos y espacios."""
    if not texto: return ""
    # Reemplazar múltiples saltos de línea por uno solo
    texto = re.sub(r'\n+', ' ', texto)
    # Quitar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

def obtener_frases_random(ruta_archivo, num_muestras=10, min_chars=50):
    """Carga un archivo, lo limpia y extrae frases al azar."""
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        contenido = f.read()

    # Dividimos por puntos para simular frases (una aproximación simple)
    # Opcional: usar nltk.sent_tokenize si quieres algo más pro
    frases = contenido.split('.')

    # Filtramos frases muy cortas (basura)
    frases_validas = [limpiar_texto(f) for f in frases if len(f) > min_chars]

    if len(frases_validas) < num_muestras:
        return frases_validas

    return random.sample(frases_validas, num_muestras)

def calcular_perplexity(model, tokenizer, lista_frases):
    """Calcula la PPL promedio de una lista de frases."""
    model.eval()
    total_loss = 0
    count = 0

    with torch.no_grad():
        for frase in lista_frases:
            inputs = tokenizer(frase, return_tensors="pt")
            if torch.cuda.is_available():
                inputs = inputs.to("cuda")

            # Calculamos loss
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss

            if not math.isnan(loss.item()):
                total_loss += loss.item()
                count += 1

    if count == 0: return 0
    avg_loss = total_loss / count
    return math.exp(avg_loss)

In [ ]:
print("--- Cargando y muestreando textos ---")
muestras_shakespeare = obtener_frases_random(PATH_TXT_SHAKESPEARE, num_muestras=50)
muestras_random      = obtener_frases_random(PATH_TXT_RANDOM, num_muestras=50)

print(f"Muestras Shakespeare: {len(muestras_shakespeare)}")
print(f"Muestras Random: {len(muestras_random)}")
print(f"Ejemplo Shakespeare: {muestras_shakespeare[0][:100]}...")
print(f"Ejemplo Corpus general: {muestras_random[0][:100]}...")

--- Cargando y muestreando textos ---
Muestras Shakespeare: 50
Muestras Random: 50
Ejemplo Shakespeare: Ah, would the scandal vanish with my life, How happy then were my ensuing death! Enter King Richard ...
Ejemplo Corpus general: Then he said he believes in us , he loves us and we 're going to win the game...


In [ ]:
def evaluar_modelo(nombre_modelo, ruta_modelo):
    print(f"\n============================================")
    print(f"CARGANDO MODELO: {nombre_modelo}")
    print(f"Ruta: {ruta_modelo}")
    print(f"============================================")

    # Cargar Tokenizer y Modelo en 4-bit para ahorrar RAM
    try:
        tokenizer = AutoTokenizer.from_pretrained(ruta_modelo, trust_remote_code=True)
        # Asegurarse de tener token de padding
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        model = AutoModelForCausalLM.from_pretrained(
            ruta_modelo,
            load_in_4bit=True,
            device_map="auto",
            torch_dtype=torch.float16
        )
    except Exception as e:
        print(f"Error cargando modelo: {e}")
        return

    # Medir
    print("-> Midiendo PPL en corpus Shakespeare...")
    ppl_shak = calcular_perplexity(model, tokenizer, muestras_shakespeare)

    print("-> Midiendo PPL en corpus No-Shakespeare...")
    ppl_rand = calcular_perplexity(model, tokenizer, muestras_random)

    print(f"\nRESULTADOS {nombre_modelo}:")
    print(f"Perplexity (Shakespeare): {ppl_shak:.4f}")
    print(f"Perplexity (Random/Moderno): {ppl_rand:.4f}")

    # LIMPIEZA DE MEMORIA (CRUCIAL EN COLAB)
    del model
    del tokenizer
    torch.cuda.empty_cache()
    gc.collect()
    print("Memoria liberada.")

In [ ]:
# 1. Evaluar Modelo Base (Sin Fine-tune)
evaluar_modelo("MISTRAL BASE", PATH_MODELO_BASE)


CARGANDO MODELO: MISTRAL BASE
Ruta: /content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-7b-instruct-v0.3


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

-> Midiendo PPL en corpus Shakespeare...
-> Midiendo PPL en corpus No-Shakespeare...

RESULTADOS MISTRAL BASE:
Perplexity (Shakespeare): 76.1964
Perplexity (Random/Moderno): 38.1371
Memoria liberada.


In [ ]:
# 2. Evaluar Modelo Fine-Tuned
evaluar_modelo("MISTRAL FINE-TUNED", PATH_MODELO_FINETUNE)


CARGANDO MODELO: MISTRAL FINE-TUNED
Ruta: /content/drive/MyDrive/StoryWriter/Modelo_FineTuning/modelo-finetuneado(lora)
Error cargando modelo: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/StoryWriter/Modelo_FineTuning/modelo-finetuneado(lora)'. Use `repo_type` argument if needed.


In [ ]:
!pip install -q peft

In [ ]:
from peft import PeftModel # <--- Importante: Necesitamos esto

def evaluar_modelo(nombre_modelo, ruta_modelo, es_adapter=False, ruta_base=None):
    print(f"\n============================================")
    print(f"CARGANDO MODELO: {nombre_modelo}")
    print(f"Ruta: {ruta_modelo}")
    if es_adapter:
        print(f"(Cargando como adaptador LoRA sobre base: {ruta_base})")
    print(f"============================================")

    try:
        if es_adapter:
            # CASO 1: Cargar Adaptador (LoRA)
            # a) Cargamos la base primero
            base_model = AutoModelForCausalLM.from_pretrained(
                ruta_base,
                load_in_4bit=True,
                device_map="auto",
                torch_dtype=torch.float16
            )
            # b) Le cargamos el adaptador encima
            model = PeftModel.from_pretrained(base_model, ruta_modelo)

            # Usamos el tokenizer de la base (generalmente es el mismo)
            tokenizer = AutoTokenizer.from_pretrained(ruta_base, trust_remote_code=True)

        else:
            # CASO 2: Cargar Modelo Completo (Base)
            tokenizer = AutoTokenizer.from_pretrained(ruta_modelo, trust_remote_code=True)
            model = AutoModelForCausalLM.from_pretrained(
                ruta_modelo,
                load_in_4bit=True,
                device_map="auto",
                torch_dtype=torch.float16
            )

        # Asegurarse de tener token de padding
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

    except Exception as e:
        print(f"Error cargando modelo: {e}")
        return

    # Medir
    print("-> Midiendo PPL en corpus Shakespeare...")
    ppl_shak = calcular_perplexity(model, tokenizer, muestras_shakespeare)

    print("-> Midiendo PPL en corpus No-Shakespeare...")
    ppl_rand = calcular_perplexity(model, tokenizer, muestras_random)

    print(f"\nRESULTADOS {nombre_modelo}:")
    print(f"Perplexity (Shakespeare): {ppl_shak:.4f}")
    print(f"Perplexity (Random/Moderno): {ppl_rand:.4f}")

    # LIMPIEZA DE MEMORIA
    del model
    del tokenizer
    if es_adapter and 'base_model' in locals():
        del base_model

    torch.cuda.empty_cache()
    gc.collect()
    print("Memoria liberada.")

# --- EJECUCIÓN ---

# 1. Evaluar Modelo Base (Igual que antes)
evaluar_modelo("MISTRAL BASE", PATH_MODELO_BASE, es_adapter=False)

# 2. Evaluar Modelo Fine-Tuned (AQUÍ ESTÁ EL CAMBIO)
# Pasamos la ruta del adaptador, activamos es_adapter=True y le decimos dónde está la base
evaluar_modelo(
    "MISTRAL FINE-TUNED (LoRA)",
    PATH_MODELO_FINETUNE,
    es_adapter=True,
    ruta_base=PATH_MODELO_BASE
)


CARGANDO MODELO: MISTRAL BASE
Ruta: /content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-7b-instruct-v0.3


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

-> Midiendo PPL en corpus Shakespeare...
-> Midiendo PPL en corpus No-Shakespeare...

RESULTADOS MISTRAL BASE:
Perplexity (Shakespeare): 76.1964
Perplexity (Random/Moderno): 38.1371


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Memoria liberada.

CARGANDO MODELO: MISTRAL FINE-TUNED (LoRA)
Ruta: /content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-finetuneado(lora)
(Cargando como adaptador LoRA sobre base: /content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-7b-instruct-v0.3)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

-> Midiendo PPL en corpus Shakespeare...
-> Midiendo PPL en corpus No-Shakespeare...

RESULTADOS MISTRAL FINE-TUNED (LoRA):
Perplexity (Shakespeare): 31.2709
Perplexity (Random/Moderno): 37.1936
Memoria liberada.
